In [1]:

using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials

In [2]:

# import conventions
include("conventions.jl")
  using .conventions: big_endian, qubit_begin
# import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate

Load quantum gates constructor


In [3]:
include("lib_tensor/QTensor.jl")
using ..QTensor: Qgate_T2D

Load Tensor module: QTensor.jl
Load quantum gates constructor


In [ ]:
#include("lib_qcircuit/MK_qcircuit.jl")


In [4]:
include("quantum_circuit.jl")
using ..quantum_circuit: qc_initialize, init_register, show_statevector, apply_op
#using ..quantum_circuit: qc_initialize, init_register, print_initstate

Load quantum gates constructor
Load quantum gates constructor


### quantum sensor test 

In [ ]:

Base.@kwdef mutable struct qc_initstruct
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector::Array{Float64,1}
    q_states::Array{Int64,1}
end # end qc_initialize


In [60]:

Base.@kwdef mutable struct qc_initstruct_test14
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector::Array{Float64,1}
    q_states::Array{Float64,1}
end # end qc_initialize


qc_initstruct_test14

In [80]:

Base.@kwdef mutable struct qc_initstruct_test21
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector
    q_states
end # end qc_initialize


qc_initstruct_test21

In [82]:

Base.@kwdef mutable struct qc_initstruct
    # Initialize the quantum register
    n_qubits::Int64
    q_order::String
    n_bas::Int64
    n_dim::Int64
    state_vector::Union{Vector{Float64},Vector{Int64}, Vector{ComplexF64}}
#    state_vector::Array{Float64,1}
    q_states
#    q_states::Array{Float64,2}
end # end qc_initialize

qc_initstruct

In [86]:
n = 3
c_sv = nothing
n_bas = 2 # number of basis states
n_qubits = n # number of qubits
n_dim = 2^n # dimensions of the quantum register/Hilbert space
state_vector = zeros(2^n) # initialize the state_vector
# create the default statevector of the quantum register:
state_vector[1] = 1.0 # set the initial state to |0>
q_states = zeros(2^n, n) # initialize the quantum states
q_tab = [0;1] # initialize the basis vectors quantum table
if n_qubits == 1
    q_states = q_tab # basis vectors
    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end
end # end if n_qubits == 1
q_order::String="big-endian"
typeof(state_vector)
#state_vector
QC_test20 = qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector,q_states)
#QC_test = qc_initstruct_test5(n_qubits, q_order, n_bas, n_dim, state_vector,q_states)

qc_initstruct(3, "big-endian", 2, 8, [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0])

In [23]:
state_vector

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [ ]:
n = 1
c_sv = nothing
n_bas = 2 # number of basis states
n_qubits = n # number of qubits
n_dim = 2^n # dimensions of the quantum register/Hilbert space
state_vector = zeros(2^n) # initialize the state_vector
# create the default statevector of the quantum register:
state_vector[1] = 1 # set the initial state to |0>
q_states = zeros(2^n, n) # initialize the quantum states
q_tab = [0;1] # initialize the basis vectors quantum table
if n_qubits == 1
    q_states = q_tab # basis vectors
    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end
end # end if n_qubits == 1

if n_qubits > 1
    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1.0 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 
end
q_order::String="big-endian"
typeof(state_vector)
#state_vector
QC_test = qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

In [ ]:

function qc_initialize_test(n::Int64, 
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol,
    q_order::String="big-endian")
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end
        
   else
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 
end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

#    return 1

end # end qc_initialize  


In [ ]:
qubit_control = 0
qubit_target = 1
nqubits = 1
err_tol = 1e-8
qc = qc_initialize_test(nqubits)


### ===============================

In [ ]:
# Base.@kwdef mutable struct qc_initstruct
#     # Initialize the quantum register
#     n::Int64
#     q_order::String
#     n_bas::Int64
#     n_dim::Int64
#     state_vector::Array{Float64,1}
#     q_states::Array{Float64,2}
# end # end qc_initialize

## refs

https://quantumcomputing.stackexchange.com/questions/14066/how-do-i-apply-the-hadamard-gate-to-one-qubit-in-a-two-qubit-pure-state


In [ ]:

function qc_initialize(n::Int64, 
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol,
    q_order::String="big-endian")
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    ## notes:
    # the minimum number of qubits is 2 

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

    return qc_initstruct(n, q_order, n_bas, n_dim, state_vector, q_states)
end # end qc_initialize  

In [ ]:
function apply_op_test(qc, Qgate)
    # Apply a quantum gate to the quantum register
    # qc::quantum register
    # gate::Qgate: quantum gate
    # return: quantum register with the quantum gate applied
    Nqubits = qc.n_qubits
    Nstates = qc.n_dim
    state_vector = qc.state_vector
    # First check if the dimensions of the quantum gate are the same 
    Qgate_dim = size(Qgate)
    if Qgate_dim[1] != Qgate_dim[2]
        error("The quantum gate is not square")
    end # end if
    # check if the quantum gate is unitary
    if ishermitian(Qgate) == false
        error("The quantum gate is not unitary")
    end # end if
    # check if the dimensions of the quantum gate 
    # ... and the quantum register do match 
    if Qgate_dim[1] != Nstates
        error("The quantum gate and the quantum register do not match")
    end # end if
    # Apply the quantum gate to the quantum register
    state_vector = Qgate * state_vector
    qc.state_vector = state_vector
    return qc
end # end apply_gate!

In [ ]:

# Type 1: tensor product of a 2D quantum gate acting on a qubit
function Qgate_T2Dtest(gate::Union{Array{Float64}, Array{Int64}, Array{ComplexF64}}, 
    qubit_target::Int64, nqubits::Int64,
    big_endian::Bool=conventions.big_endian,
    err_tol::Float64=err_tol)
# gate_qn is used to construct the quantum gate acting on the qubit i 
# ... of a quantum register of nqubits qubits
# gate is a reduced representation of the quantum gate: minimal representation 
# ... of the quantum gate in 2x2 matrices, 4x4 matrices, etc.
# qubit_target is the index of the qubit on which the gate acts
# nqubits is the number of qubits in the quantum register
# big_endian is a boolean variable that indicates whether the qubits are
# ... ordered in big endian or little endian
# Return statevector after the action of the operator 
# First construct the array of the qubits 
# ... in the quantum register
qubits = Array{Int64}(undef, nqubits)
qubits = collect(0:nqubits-1)

# check the convention of the index_start where qubit counting can start from 0 or 1
# ... the default is 0.
# nqubits is the number of qubits in the quantum register
#if !qubit_start_1
# qubit_end = nqubits # number of qubits in the quantum register
#  qubit_begin = 1
#else
qubit_end = nqubits-1 # number of qubits in the quantum register
qubit_begin = 0
#end
# check if qubit_target is an integer, and in range
if !isa(qubit_target, Int64) || qubit_target <  qubit_begin || qubit_target > nqubits-1
error("The target qubit must be an integer in range: ", qubit_begin, " ", qubit_end)
end
# the tensor product used to construct the quantum gate 
# ... acting on the qubit qubit_index is independent of the convention. 
gate_construct = 1
II = Matrix(I, 2, 2)
if big_endian
for i in  qubit_begin:qubit_end
if i == qubit_target
gate_construct = kron(gate,gate_construct)
else
gate_construct = kron(gate_construct,II)
end
end
else
for i in  qubit_end:-1:qubit_begin
if i == qubit_target
gate_construct = kron(gate,gate_construct)
else
gate_construct = kron(gate_construct,II)
end
end
end
return gate_construct
# note: the little-endian convention is not tested yet
end # Qgate_T2D

In [ ]:

function qc_initialize_test(n::Int64, 
    c_sv= nothing, 
#     c_sv::Union{Vector{Float64}, Vector{Int64}, Vector{ComplexF64}} = nothing,     
    err_tol::Float64=err_tol,
    q_order::String="big-endian")
    # Initialize the quantum register
    # n::Int64: number of qubits
    # q_order::String: order of the qubits in the quantum register
    # return: quantum register of n qubits
    # q_order == "big-endian"
    # q_order == "little-endian"

    # start the function
    n_bas = 2 # number of basis states
    n_qubits = n # number of qubits
    n_dim = 2^n # dimensions of the quantum register/Hilbert space
    state_vector = zeros(2^n) # initialize the state_vector
    # create the default statevector of the quantum register: 
    state_vector[1] = 1 # set the initial state to |000 ...0>
    q_states = zeros(2^n, n) # initialize the quantum states
    q_tab = [0;1] # initialize the basis vectors quantum table
   

    ## notes:
    # the minimum number of qubits is 2 
    if n_qubits == 1
        q_states = q_tab # basis vectors
        # check if the user has provided a custom statevector
        # err_tol = 1e-16 # error tolerance for checking the unitary condition
        if c_sv != nothing
            # check if the custom statevector has the correct dimensions
            if length(c_sv) != n_dim
                error("The custom statevector has the wrong dimensions")
            end
            if isapprox(norm(c_sv), 1, rtol=err_tol) == false
                error("The custom statevector is not normalized")
            end
            state_vector = c_sv
        end
        
   else
    n_count = n_bas
    for i = 2:n_qubits
        q_tab = [zeros(n_count,1) q_tab
                ones(n_count,1) q_tab]
        n_count = n_count*2
    end
    q_states = q_tab # basis vectors 
end # end if n_qubits == 1

    # check if the user has provided a custom statevector
    # err_tol = 1e-16 # error tolerance for checking the unitary condition
    if c_sv != nothing
        # check if the custom statevector has the correct dimensions
        if length(c_sv) != n_dim
            error("The custom statevector has the wrong dimensions")
        end
        if isapprox(norm(c_sv), 1, rtol=err_tol) == false
            error("The custom statevector is not normalized")
        end
        state_vector = c_sv
    end

    # one final check for the statevector
    if isapprox(norm(state_vector), 1, rtol=err_tol) == false
        error("The statevector is not normalized")
    end

#    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, 1.0, [1.0 1.0])
    return qc_initstruct(n_qubits, q_order, n_bas, n_dim, state_vector, q_states)

#    return 1

end # end qc_initialize  
